# 第9章: ベクトル空間法(1)

### 80. コーパスの整形

* 1/10サンプリングデータ

In [1]:
with open("enwiki-20150112-400-r10-105752.txt") as fr, \
      open("EWikiCorpus.txt", mode="w") as fw:
    for line in fr:
        tokens = []
        line = line.rstrip()
        for chunk in line.split():
            token = chunk.strip().strip('.,!?;:()[]\'"')
            if len(token) > 0:
                tokens.append(token)
        print(*tokens, sep=" ", end="\n", file=fw)

* 1/100サンプリングデータ

In [2]:
with open("enwiki-20150112-400-r100-10576.txt") as fr, \
      open("EWikiCorpusMini.txt", mode="w") as fw:
    for line in fr:
        tokens = []
        line = line.rstrip()
        for chunk in line.split():
            token = chunk.strip().strip('.,!?;:()[]\'"')
            if len(token) > 0:
                tokens.append(token)
        print(*tokens, sep=" ", end="\n", file=fw)

### 81. 複合語からなる国名への対処

In [3]:
#Webスクレイピングでネットから世界の国名リストをもらってくる
import requests
from bs4 import BeautifulSoup
target_url = "https://www.nationsonline.org/oneworld/countries_of_the_world.htm"
r = requests.get(target_url)
soup = BeautifulSoup(r.text, "lxml")

In [4]:
#取得したスープから国のリストを生成
import re
country_list = []
for tr in soup.find_all("tr"):
    country = []
    for td in tr.find_all("td", class_=("tdb", "tdx")):
        country.append(td.string)
    if len(country) > 0:
        country_name = str(country[0]).split()
        if len(country_name) > 1:
            country_list.append(" ".join(country_name))
            
country_list.remove("Cocos (Keeling) Islands")
country_list.remove("Iran (Islamic Republic of)")
country_list.remove("Korea, Democratic People's Rep. (North Korea)")
country_list.remove("Korea, Republic of (South Korea)")
country_list.remove("Lao, People's Democratic Republic")
country_list.remove("Macedonia, Rep. of")
country_list.remove('Micronesia, Federal States of')
country_list.remove("Moldova, Republic of")
country_list.remove("Slovakia (Slovak Republic)")
country_list.remove('Vatican City State (Holy See)')
country_list.remove("Virgin Islands (British)")
country_list.remove("Virgin Islands (U.S.)")
country_list.append("Cocos Islands")
country_list.append("North Korea")
country_list.append("South Korea")
country_list.append("Slovak Republic")
country_list.append("Vatican City State")
country_list.append("Virgin Islands")
country_list.append("Isle of Man")

with open("CountryList.txt", mode="w") as fw:
    for country in country_list:
        print(country, file=fw)

In [5]:
country_dict = {country.split()[0]: country for country in country_list}
print(country_dict)

{'American': 'American Samoa', 'Antigua': 'Antigua and Barbuda', 'Bosnia': 'Bosnia and Herzegovina', 'Brunei': 'Brunei Darussalam', 'Burkina': 'Burkina Faso', 'Cape': 'Cape Verde', 'Cayman': 'Cayman Islands', 'Central': 'Central African Republic', 'Christmas': 'Christmas Island', 'Cook': 'Cook Islands', 'Costa': 'Costa Rica', 'Czech': 'Czech Republic', 'El': 'El Salvador', 'Equatorial': 'Equatorial Guinea', 'Falkland': 'Falkland Islands', 'Faroe': 'Faroe Islands', 'French': 'French Southern Territories', 'Great': 'Great Britain', 'Holy': 'Holy See', 'Hong': 'Hong Kong', 'Ivory': 'Ivory Coast', 'Marshall': 'Marshall Islands', 'Netherlands': 'Netherlands Antilles', 'New': 'New Zealand', 'Northern': 'Northern Mariana Islands', 'Palestinian': 'Palestinian territories', 'Papua': 'Papua New Guinea', 'Pitcairn': 'Pitcairn Island', 'Puerto': 'Puerto Rico', 'Reunion': 'Reunion Island', 'Russian': 'Russian Federation', 'Saint': 'Saint Vincent and the Grenadines', 'San': 'San Marino', 'Sao': 'Sao

In [6]:
with open("EWikiCorpusMini.txt") as fr, \
      open("EMiniMod.txt", mode="w") as fw:
    for line in fr:
        line = line.rstrip()
        line = line.split()
        for wordindex, oneword in enumerate(line):
            if oneword in country_dict.keys():
                if " ".join(line[wordindex:(wordindex+len(country_dict[oneword])-1)]) == country_dict[oneword]:
                    new_word = "_".join(line[wordindex:(wordindex+len(country_dict[oneword])-1)])
                    del line[wordindex:(wordindex+len(country_dict[oneword])-1)]
                    line.insert(wordindex, new_word)
        print(" ".join(line), file=fw)

### 82. 文脈の抽出

In [24]:
import random
with open("EMiniMod.txt") as fr, \
      open("EMiniContext.txt", mode="w") as fw:
    for line in fr:
        line = line.rstrip()
        line = line.split()
        for t_index in range(len(line)):
            d = random.randint(1, 5)
            for i in range(max(t_index-d, 0), min(t_index+d+1, len(line))):
                if i != t_index:
                    print(line[t_index] + "\t" + line[i], file=fw)

### 83. 単語/文脈の頻度の計測

In [31]:
from collections import Counter

with open("EMiniContext.txt") as fr:
    list_tc = []
    list_t = []
    list_c = []
    N = 0
    for line in fr:
        line = line.rstrip()
        tokens = line.split("\t")
        list_tc.append(line)
        list_t.append(tokens[0])
        list_c.append(tokens[1])
        N += 1

N -= 1
print("N: {}".format(N))
Count_tc = Counter(list_tc)
Count_t = Counter(list_t)
Count_c = Counter(list_c)

N: 68153346


In [32]:
%store N
%store Count_tc
%store Count_t
%store Count_c

Stored 'N' (int)
Stored 'Count_tc' (Counter)
Stored 'Count_t' (Counter)
Stored 'Count_c' (Counter)


### 84. 単語文脈行列の作成

In [1]:
%store -r N
%store -r Count_tc
%store -r Count_t
%store -r Count_c

In [2]:
from scipy.sparse import lil_matrix
from collections import OrderedDict

t_index = OrderedDict((key, i) for i, key in enumerate(Count_t.keys()))
c_index = OrderedDict((key, i) for i, key in enumerate(Count_c.keys()))

In [7]:
import numpy as np

X = lil_matrix((len(Count_t), len(Count_c)))

for key in Count_tc.keys():
    if Count_tc[key] >= 10:
        t, c = key.split("\t")
        ppmi = max(np.log((N*Count_tc[key]) / (Count_t[t]*Count_c[c])), 0)
        X[t_index[t], c_index[c]] = ppmi

In [9]:
%store X

Stored 'X' (lil_matrix)


In [3]:
%store -r X

### 85. 主成分分析による次元圧縮

In [4]:
from sklearn.decomposition import TruncatedSVD

pca = TruncatedSVD(n_components = 300)
X_pca = pca.fit_transform(X)
print("original X shape: {}".format(X.shape))
print("X_pca shape:{}".format(X_pca.shape))

original X shape: (383307, 383307)
X_pca shape:(383307, 300)


In [5]:
%store X_pca

Stored 'X_pca' (ndarray)


In [ ]:
%store -r X_pca

### 86. 単語ベクトルの表示

In [17]:
print(X_pca[t_index["United_States"]])

[ 2.79199585e-01 -7.92988847e-02 -5.30843002e-01  3.57516690e-01
 -6.41936835e-02  3.97280914e-01  1.63381849e-02 -2.80473779e-01
 -5.57055979e-02 -2.48698978e-01 -7.03300722e-01  6.91704965e-02
 -5.17045439e-01 -7.72291770e-02 -6.75260409e-02 -2.86649372e-01
  2.45289175e-01  4.29554749e-02 -1.14095646e-01 -3.28966262e-01
  3.19320944e-01 -2.59354859e-01  4.87557649e-01  1.94632276e-01
 -1.77250228e-01  5.77243665e-02  2.24807974e-02 -5.86683600e-01
  2.11993848e-01 -2.62488494e-01 -1.68962551e-01 -2.74591687e-01
  2.55959399e-02 -9.37128017e-02  1.86827791e-01  1.95056082e-01
  6.75651087e-02  1.18463287e-01 -2.28338566e-02  2.11160935e-02
  1.25455810e-01  3.21649831e-01 -2.46326468e-01 -2.22650021e-01
 -3.45318267e-02 -1.68043816e-01  1.52377445e-01  3.72811445e-01
  1.46397917e-01  8.92022354e-02  1.18447301e-01 -3.05462224e-01
  5.76099460e-02  9.34595654e-02  2.50573183e-01  6.22760763e-01
 -4.62849194e-01  4.95562402e-04 -6.38552515e-02 -7.68523887e-01
  2.08025213e-01 -8.10863

### 87. 単語の類似度

In [21]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

print(cos_sim(X_pca[t_index["United_States"]], X_pca[t_index["U.S"]]))

0.03324777703015864


### 88. 類似度の高い単語10件

In [31]:
Vec_england = X_pca[t_index["England"]]

list_Vec = []
for index in t_index.values():
    temp_sim = cos_sim(Vec_england, X_pca[index])
    list_Vec.append(temp_sim)

/Users/takapen325/.pyenv/versions/anaconda3-5.2.0/envs/mainEnv/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


In [32]:
Sim_rank = np.argsort(list_Vec)[::1]
list_word = list(t_index.keys())
for index in Sim_rank[:10]:
    print(list_word[index] + "\t" + str(list_Vec[index]))

Framework	-0.09834245111896592
Asset	-0.09505085734428634
Ivor	-0.09445805971862348
Ater	-0.09445805971862346
Horthy	-0.0917532226551083
Muak	-0.09155921802754437
Duran	-0.08872148434800202
Dadullah	-0.0863716311506307
Roi	-0.0862304870031103
Ching	-0.08623027502483759


### 89. 加法構成性によるアナロジー

In [34]:
Vec_question = X_pca[t_index["Spain"]] - X_pca[t_index["Madrid"]] + X_pca[t_index["Athens"]]

list_Vec2 = []
for index in t_index.values():
    temp_sim = cos_sim(Vec_question, X_pca[index])
    list_Vec2.append(temp_sim)
    
Sim_rank2 = np.argsort(list_Vec2)[::1]
list_word = list(t_index.keys())
for index in Sim_rank2[:10]:
    print(list_word[index] + "\t" + str(list_Vec2[index]))

/Users/takapen325/.pyenv/versions/anaconda3-5.2.0/envs/mainEnv/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


Bois	-0.14715548511358012
Mundial	-0.14456570550355424
Nacional	-0.14317157626344854
Pole	-0.14301961026977175
España	-0.14280670370012225
Varzim	-0.14280670370012225
Rambaud	-0.14280670370012225
comte	-0.14280670370012225
Vaca	-0.14280670370012225
los	-0.14280670370012225
